In [1]:
import keras
import os
import json
import tensorflow as tf





Using TensorFlow backend.


In [2]:
# # """phase 1- delft 15"""

# data_dir=''
# event_dict=('delft15_phase2/all_event_dict_delft15.json')
# skill=os.path.join(data_dir+"skill_df_delft_15.csv")
# response=os.path.join(data_dir+"response_df_delft_15.csv")




"""phase 1- delft 14"""


# data_dir='../code/data_dkt/delft14_phase1/'
# skill_dict=json.load(open('data_dkt/delft15_phase2/skill_dict_delft_all_0_336_all.json'))
# skill=os.path.join(data_dir+"skill_df_delft_14_phase1.csv")
# response=os.path.join(data_dir+"response_df_delft_14_phase1.csv")

'phase 1- delft 14'

In [3]:

"""phase 2 """
# data_dir = "Delft15_phase2/"
# skill_dict=json.load(open(os.path.join(data_dir,"unique_events.json")))
# response = os.path.join(data_dir,"effort_response.csv")
# skill = os.path.join(data_dir,"skill.csv")


'phase 2 '

In [4]:

!ls ../code/data_dkt/delft15/

ls: cannot access '../code/data_dkt/delft15/': No such file or directory


# Changed processing file data_helper.py

In [5]:
 


import pandas as pd
import numpy as np
import json
import os

 
""" ONE HOT ENCODING OF SKILL AND RESPONSE DATA"""  


def load_data(skill_csv,response_csv,skill_json,problem_num,is_behaviour=False):
    """
    This function reads data files:
    1. skill_dict:reads data from __skill_dict.json__ which maps skills to numbers 1,2..
    2. skill_csv: reads data from __skill.csv__ which is a sequence of exercise or skills attempted by a student
    3. response_csv: reads data from reponse.csv which is sequence of binary response to skill/exercise in skill.csv by each student
    4. problem_num: total number of problem events in the course, each tagged from 1 to problem_num in skill_json which a dictionary that maps events to integers
    5. is_behaviour is the boolean value which is TRUE when we want to include behavior events in our model or false for phase-1 dkt

    """
    #response_df = pd.read_csv('correct.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # data_dir="data/dktData/"
    data_dir=""
    response_csv = os.path.join(data_dir,response_csv)
    # response_df = pd.read_csv(response_csv, sep='\t').drop('Unnamed: 0', axis=1)
    response_df = pd.read_csv(response_csv)
    skill_csv = os.path.join(data_dir,skill_csv)
    skill_df=pd.read_csv(skill_csv)
    skill_dict=json.load(open(skill_json))


     
#   1. skill matrix
    skill_matrix = skill_df.iloc[:, 1:].values
#   2. response matrix
    response_array = response_df.iloc[:, 1:].values
#     print('skill_matrix .shape,response_array.shape')
#     print(skill_matrix .shape,response_array.shape)
    if is_behaviour is False:
        non_problem_event_num=0
        skill_matrix[skill_matrix>problem_num]=0
    else:
        non_problem_event_num=len(skill_dict)-problem_num
    return skill_matrix,response_array,problem_num, non_problem_event_num


def preprocess(skill_matrix,response_array,problem_num=336, non_problem_event_num=0):
    """
    This function extracts the skills and responses from the loaded files excluding student ids
    :param skill_df: skills attempted by a student at each timestep
    :param response_df: responses on the exercises  by a student at each timestep
    :param problem_num: Total number of problem events in the course
    :param non_problem_num: Total number of non-problem events 
    """


#     print('CHECK:::NUMBER of NON- problem events, problem events :,', non_problem_event_num,problem_num)
    
    
#   1. skillarray
    skill_array =convert_event_to_one_hot(skill_matrix, problem_num+non_problem_event_num)
      
#  2. skill_response array
    skill_response_array = append_response_one_hot(skill_array,skill_matrix, response_array,problem_num)
    
#     print('skill_array.shape,response_array.shape,skill_response_array.shape')
    print(skill_array.shape,response_array.shape,skill_response_array.shape)
    
    return skill_array, response_array, skill_response_array




def convert_event_to_one_hot(skill_matrix, vocab_size):
    # 1. Create one hot encode row for each integer from 0 to vocab_size
    print(vocab_size)
    one_hot_dict = np.eye(vocab_size) 
    # 2. TO ASSIGN [000] to all out of vocab_size numbers, add last row with all zeros
    one_hot_dict = np.vstack((one_hot_dict,np.zeros(vocab_size)))
#     ADDED ONE MORE ROW SO THAT  IF WE FILL NA WITH -1 then -1-1= -2 ,
# so 2nd last row becomes the assigned value
    one_hot_dict = np.vstack((one_hot_dict,np.zeros(vocab_size)))
    
    
    # sequence length is the number timesteps in the sequence
    sequence_len = skill_matrix.shape[1] 
    # instantiation of a numpy array with all elements 0
    on_hot_sequence = np.empty((skill_matrix.shape[0],sequence_len, vocab_size))

    for row in range(skill_matrix.shape[0]):
        # set vocabulary values in skills sequence of a student equal to 1, index of skills starts at 1
        on_hot_sequence[row] = one_hot_dict [skill_matrix[row]-1] #grab 1-hot rows for integers in skill_matrix
#     print('Check encode in skill and one hot skill:', len(skill_matrix[skill_matrix!=0]),np.sum( on_hot_sequence))
    return on_hot_sequence




def append_response_one_hot(skill_array,skill_matrix, response_matrix, problem_num):
    """
   params:
       skill_matrix: 2-D matrix (student, skills)
       response_matrix:  2-D matrix (student, responses)
    with_response_vocab_size: Number of (2*problem events AND non-problem events) in the course
   returns:
       a 3d-darray with a shape like (student, sequence_len,problem_num)
   """
    # 1. Create one hot encode row for each integer from 0 to problem_num
    one_hot_dict_problems_only = np.eye(problem_num) 
    # 2. TO ASSIGN [000] to all out of vocab_size numbers
    one_hot_dict_problems_only= np.vstack((one_hot_dict_problems_only,np.zeros(problem_num)))
    #     ADDED ONE MORE ROW SO THAT  IF WE FILL NA WITH -1 then -1-1= -2 ,
# so 2nd last row becomes the assigned value
    one_hot_dict_problems_only= np.vstack((one_hot_dict_problems_only,np.zeros(problem_num))) 
    # 3. Get problem  ids of problems that are correct so that they can be assigned 1 value in one hot encode
#     all other  are 0, - ----this is why we encode are not encoding  any skill as  ----
    skill_matrix_only_correct_problems=skill_matrix*response_matrix

  
    # sequence length is the number timesteps in the sequence
    sequence_len = skill_matrix.shape[1] 
    # instantiation of a numpy array with all elements 0
    response_one_hot = np.empty((skill_matrix.shape[0],sequence_len, problem_num))
    # iterate over each student in data
    for i in range(response_matrix.shape[0]):
        # set vocabulary values in skills attempted by a student equal to 1
        # get encodes for sequences with non-zero problem
        response_one_hot[i]=one_hot_dict_problems_only[skill_matrix_only_correct_problems[i]-1]
        
#     print('Check number correct in response and one hot response:', np.sum(response_matrix),np.sum(response_one_hot))
    skill_response_array=np.concatenate((skill_array,response_one_hot),axis=2)
#     print('skill_array.shape,response_one_hot.shape,skill_response_array.shape')
#     print(skill_array.shape,response_one_hot.shape,skill_response_array.shape)
    return  skill_response_array








# DKT.PY

In [6]:
from keras.models import Model
from keras.layers import Input, Dropout, Masking, Dense, Embedding
from keras.layers import Embedding
from keras.layers.core import Flatten, Reshape
from keras.layers import LSTM
from keras.layers import merge
from keras.layers.merge import multiply
from keras.callbacks import EarlyStopping
from keras import backend as K
from theano import tensor as T
from theano import config
from theano import printing
from theano import function
from keras.layers import Lambda
import theano
import numpy as np
import pdb
from math import sqrt
from keras.callbacks import Callback



class DKTcustomloss():

    def __init__(self, 
                input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=[], 
                y_train=[], 
                y_train_order=[],
                validation_split=0.0,
                validation_data=None,
                optimizer='adam',
                callbacks=None):
        """

        :param input_dim: dimension of the input at one timestamp (dimension of x_t= 2*num_skills)
        :param input_dim_order: dimension of the one-hot representation of problem to check order of occurence(=num_skills)
        :param hidden_layer_size: number of nodes in hidden layer
        :param x_train: 3D matrix of size (samples, number of timestamp/sequence length, dimension of input vec (x_t) )
        :param y_train: a matrix of responses (samples,number of timestamp/sequence length)
        :param y_train_order: shape of output equal to number of timesteps
        :param validation_split:
        :param validation_data:
        :param optimizer:
        :param callbacks:

        """
        ## input dim is the dimension of the input at one timestamp (dimension of x_t)
        self.input_dim = int(input_dim) #2* num_skills

        ## input_dim_order is the dimension of the one-hot representation of problem
        ## CHECK: order of occurence of responses should be according to timestamp
        self.input_dim_order = int(input_dim_order)#num_skills

        self.hidden_layer_size = int(hidden_layer_size)
        self.batch_size = int(batch_size)
        self.epochs = int(epochs)

        ## x_train is a 3D matrix of size (samples, number of timestamp, dimension of input vec (x_t) )
        ## in cognitive tutor # of students * # total responses * # input_dim
        self.x_train = x_train
        ## y_train is a matrix of (samples one hot representation according to problem output value at each timestamp (y_t) )
        self.y_train = y_train
        ## y_train_order is the one hot representation of problem according to timestamp starting from
        ## t=1 if training starts at t=0
        self.y_train_order = y_train_order
        # users: no of student datapoints
        self.users = np.shape(x_train)[0]
        self.validation_split = validation_split
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.callbacks = callbacks
        print ("DKTnet initialization done")
        
        

        
    def LossModel(y_train,dense_out):

        dense_output= Input(batch_shape=(None, None, self.input_dim_order), name='predicted_prob')
        y_order=Input(batch_shape=(None, None, self.input_dim_order), name='skill_array')
   
        merged = multiply([dense_out, y_order])
         ## 6. Chooses the max value(which is a non zero number in merged) from the output of merged layer
        # this will reduce dimension from skill_num to 1 for each timestep

        reduced = Lambda(reduce_dim, output_shape=reduce_dim_shape)(merged)
        cce = tf.losses.softmax_cross_entropy(self.y_train,reduced)
        loss_m = Model(inputs=[dense_output,y_order], outputs=cce)
        #it's important to make this model not trainable if it has weights 
        #(you should probably set these weights manually if that's the case)    
        loss_m.trainable = False
        return loss_m

    def build_train_on_batch(self):
            ## 1. First layer for the input (x_t), creates a tensor object
            x = Input(batch_shape=(None, None, self.input_dim), name='x')

            ## 2. Mask unknown or anomalous valued timesteps in x
            # the timestep will be masked (skipped) if all values in the input tensor
            #  at that timestep are equal to mask_value
            masked = Masking(mask_value=0)(x)

            ## 3. Add a lstm layer, return sequences is True to allow output have same
            # dimension as number of timesteps in input
            lstm_out = LSTM(self.hidden_layer_size, return_sequences=True)(masked)

            ## 4. Add a fully connected layer on lstm layer, 
            ### this gives us probabilities of all events at differnt timesteps
            dense_out = Dense(self.input_dim_order, activation='sigmoid')(lstm_out)
            

            
            
            def get_probability_of_timestep_event(x):
                #Custom tensor arithmatic from backend K
                # chooses the max value from the output of previous layer
                # this will reduce dimension from one hot of skill_num to 1 for each timestep
                x = K.max(x, axis=2, keepdims=True)
                return x

        
             ## 5.1 Get problem event encoding ONLY: 
            #  ASSUMPTION:that skill dict has problem events only from keys 1:input_dim_order
            
            def get_skill_array(x):
                return x[:,:,:self.input_dim_order]
  
            
#             y_order=Lambda(get_skill_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
            
#             ## 5.2 In dense output only retain probability of event at that timestep all others 0
#             merged = multiply([dense_out, y_order])

#             ## 6. Chooses the max value from the output of merged which is the prob
#                 # this will reduce dimension from skill_num to 1 for each timestep

#             reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)
    
#             ## 7. Creates model object with specified input and output
#             self.model = Model(inputs=x, outputs=reduced)

#             ## 8. Compile model by assigning loss function for backpropagtion
#             self.model.compile(optimizer=self.optimizer,
#                                loss='binary_crossentropy',
#                                metrics=['accuracy'])

#             print('Summary of the model')
#             self.model.summary()


            
            def custom_loss(y_train,dense_out):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                print("using custom loss.....")

                
                y_order=Lambda(get_skill_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
            

                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([dense_out, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)

                cce = K.mean(K.binary_crossentropy(y_train, reduced ))#, axis=-1)
                return cce
            
            def accuracy2(y_train,dense_out):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                print("using custom loss.....")

                
                y_order=Lambda(get_skill_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
            

                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([dense_out, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)

                return K.mean(K.equal(y_train, K.round(reduced)))


            
            def custom_metric_accuracy(y_train,dense_out):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                print("using custom metric.....")
                def get_response_array(x):
                    return x[:,:,-self.input_dim_order:]
                
                
                
                actual_response=Lambda(get_response_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
                print(actual_response.get_shape(),"actual response")
                def idx_max_prob(x):
                    max_idx= K.argmax(x,axis=-1)
#                         convert to one hot, to tackle,incorrect question argmax
                    one_hot_idx=tf.one_hot(max_idx,
                                self.input_dim_order,
                                on_value=1.0,
                                off_value=0.0,
                                axis=-1)
                    return one_hot_idx
        
                max_predicted_prob = Lambda(idx_max_prob,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(dense_out)
                print( max_predicted_prob.get_shape()," max_predicted_prob")
                # we use merging, we want to find how many 1-1 matchings are there
                merged_metric = multiply([actual_response,  max_predicted_prob])


                accuracy = K.mean(K.sum(merged_metric,axis=-1))
                print(accuracy.get_shape())
                return accuracy


            ## 7.Creates model object with specified input and output
                #CHANGED: OUTPUTS: TO GET probabilities at all timesteps as output.

            self.model = Model(inputs=x, outputs=dense_out)

            ## 8. Compile model by assigning loss function for backpropagtion
            self.model.compile(optimizer=self.optimizer,
                               loss=custom_loss,
                               metrics=[accuracy2])

            print('Summary of the model')
            self.model.summary()
   

    def train_on_batch(self, x_train,y_train,y_train_order):
# y_train_order: not used
        self.model.train_on_batch(x_train, y_train)




    def test_on_batch(self, x_val, y_val,y_train_order):
        
        # y_train_order: not used
        """
       Test the model on a single batch of samples
       :return: Scalar test loss (if the model has a single output and no metrics) or list of scalars (if the model has multiple outputs and/or metrics).
       The attribute model.metrics_names will give you the display labels for the scalar outputs.
       """
#         print(self.model.metrics_names)
        return self.model.test_on_batch(x_val, y_val)

    def predict(self, x_val,y_train_order):
#              # y_train_order: not used
        return self.model.predict_on_batch(x_val)




class DKTnet():

    def __init__(self, 
                input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=[], 
                y_train=[], 
                y_train_order=[],
                validation_split=0.0,
                validation_data=None,
                optimizer='adam',
                callbacks=None):
        """

        :param input_dim: dimension of the input at one timestamp (dimension of x_t= 2*num_skills)
        :param input_dim_order: dimension of the one-hot representation of problem to check order of occurence(=num_skills)
        :param hidden_layer_size: number of nodes in hidden layer
        :param x_train: 3D matrix of size (samples, number of timestamp/sequence length, dimension of input vec (x_t) )
        :param y_train: a matrix of responses (samples,number of timestamp/sequence length)
        :param y_train_order: shape of output equal to number of timesteps
        :param validation_split:
        :param validation_data:
        :param optimizer:
        :param callbacks:

        """
        ## input dim is the dimension of the input at one timestamp (dimension of x_t)
        self.input_dim = int(input_dim) #2* num_skills

        ## input_dim_order is the dimension of the one-hot representation of problem
        ## CHECK: order of occurence of responses should be according to timestamp
        self.input_dim_order = int(input_dim_order)#num_skills

        self.hidden_layer_size = int(hidden_layer_size)
        self.batch_size = int(batch_size)
        self.epochs = int(epochs)

        ## x_train is a 3D matrix of size (samples, number of timestamp, dimension of input vec (x_t) )
        ## in cognitive tutor # of students * # total responses * # input_dim(2*NO SKILLS)
        self.x_train = x_train
        ## y_train is a matrix of (samples one hot representation according to problem output value at each timestamp (y_t) )
        self.y_train = y_train
        ## y_train_order is the one hot representation of problem according to timestamp starting from
        ## t=1 if training starts at t=0
        self.y_train_order = y_train_order
        # users: no of student datapoints
        self.users = np.shape(x_train)[0]
        self.validation_split = validation_split
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.callbacks = callbacks
        print ("DKTnet initialization done")

    def build_train_on_batch(self):
        ## 1. First layer for the input (x_t), creates a tensor object
        x = Input(batch_shape=(None, None, self.input_dim), name='x')

        ## 2. Mask unknown or anomalous valued timesteps in x
        # the timestep will be masked (skipped) if all values in the input tensor
        #  at that timestep are equal to mask_value
        masked = Masking(mask_value=-1)(x)

        ## 3. Add a lstm layer, return sequences is True to allow output have same
        # dimension as number of timesteps in input
        lstm_out = LSTM(self.hidden_layer_size, return_sequences=True)(masked)

        ## 4. Add a fully connected layer on lstm layer

        dense_out = Dense(self.input_dim_order, activation='sigmoid')(lstm_out)
        ## 5. Create  a tensor object --not sure if its required
        y_order = Input(batch_shape=(None, None, self.input_dim_order), name='y_order')
        merged = multiply([dense_out, y_order])

        def reduce_dim(x):
            #Custom tensor arithmatic from backend K
            # chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
            x = K.max(x, axis=2, keepdims=True)
            return x

        def reduce_dim_shape(input_shape):
            shape = list(input_shape)
            shape[-1] = 1
            return tuple(shape)

        ## 6. Chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
        reduced = Lambda(reduce_dim, output_shape=reduce_dim_shape)(merged)

        ## 7. Creates model object with specified input and output
        self.model = Model(inputs=[x, y_order], outputs=reduced)

        ## 8. Compile model by assigning loss function for backpropagtion
        self.model.compile(optimizer=self.optimizer,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        print('Summary of the model')
        self.model.summary()



    def train_on_batch(self, x_train, y_train, y_train_order):

        self.model.train_on_batch([x_train, y_train_order], y_train)



    def test_on_batch(self, x_val, y_val, y_val_order):
        """
       Test the model on a single batch of samples.
       :param x_train:
       :param y_train:
       :param y_train_order:
       :return: Scalar test loss (if the model has a single output and no metrics) or list of scalars (if the model has multiple outputs and/or metrics).
       The attribute model.metrics_names will give you the display labels for the scalar outputs.
       """
        print(self.model.metrics_names)
        return self.model.test_on_batch([x_val, y_val_order], y_val)



    def predict(self, x_val, y_val_order):
        return self.model.predict([x_val, y_val_order])



    
  
    


In [7]:
#testing :
from keras.models import Model
from keras.layers import Input, Dropout, Masking, Dense, Embedding
from keras.layers import Embedding


class DKTnettest():

    def __init__(self, 
                input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=[], 
                y_train=[], 
                y_train_order=[],
                validation_split=0.0,
                validation_data=None,
                optimizer='adam',
                callbacks=None,
                problem_no=336):
        """

        :param input_dim: dimension of the input at one timestamp (dimension of x_t= 2*num_skills)
        :param input_dim_order: dimension of the one-hot representation of problem to check order of occurence(=num_skills)
        :param hidden_layer_size: number of nodes in hidden layer
        :param x_train: 3D matrix of size (samples, number of timestamp/sequence length, dimension of input vec (x_t) )
        :param y_train: a matrix of responses (samples,number of timestamp/sequence length)
        :param y_train_order: shape of output equal to number of timesteps
        :param validation_split:
        :param validation_data:
        :param optimizer:
        :param callbacks:

        """
        ## input dim is the dimension of the input at one timestamp (dimension of x_t)
        self.input_dim = int(input_dim) #2* num_skills

        ## input_dim_order is the dimension of the one-hot representation of problem
        ## CHECK: order of occurence of responses should be according to timestamp
        self.input_dim_order = int(input_dim_order)#num_skills

        self.hidden_layer_size = int(hidden_layer_size)
        self.batch_size = int(batch_size)
        self.epochs = int(epochs)

        ## x_train is a 3D matrix of size (samples, number of timestamp, dimension of input vec (x_t) )
        ## in cognitive tutor # of students * # total responses * # input_dim(2*NO SKILLS)
        self.x_train = x_train
        ## y_train is a matrix of (samples one hot representation according to problem output value at each timestamp (y_t) )
        self.y_train = y_train
        ## y_train_order is the one hot representation of problem according to timestamp starting from
        ## t=1 if training starts at t=0
        self.y_train_order = y_train_order
        # users: no of student datapoints
        self.users = np.shape(x_train)[0]
        self.validation_split = validation_split
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.callbacks = callbacks
        print ("DKTnet initialization done")

    def build_train_on_batch(self):
        ## 1. First layer for the input (x_t), creates a tensor object
        x = Input(batch_shape=(None, None, self.input_dim), name='x')

        ## 2. Mask unknown or anomalous valued timesteps in x
        # the timestep will be masked (skipped) if all values in the input tensor
        #  at that timestep are equal to mask_value
        masked = Masking(mask_value=0)(x)

        ## 3. Add a lstm layer, return sequences is True to allow output have same
        # dimension as number of timesteps in input
        lstm_out = LSTM(self.hidden_layer_size, return_sequences=True)(masked)

        ## 4. Add a fully connected layer on lstm layer

        dense_out = Dense(self.input_dim_order, activation='sigmoid')(lstm_out)
       
      

        def reduce_dim(x):
            #Custom tensor arithmatic from backend K
            # chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
            x = K.max(x, axis=2, keepdims=True)
            return x

        def get_skill_array(x):
            return x[:,:,:self.input_dim_order]
        
        
        
         ## 5. Create  a tensor object --not sure if its required
#         y_order = Input(batch_shape=(None, None, self.input_dim_order), name='y_order')

#         y_order=x[:,:,:self.input_dim_order]
        y_order=Lambda(get_skill_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)

        merged = multiply([dense_out, y_order])

        ## 6. Chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
#         reduced = Lambda(reduce_dim )(merged)
        reduced = Lambda(reduce_dim , output_shape=lambda s: (s[0], s[1],1))(merged)
        print("DIM OF y_order,reduced ",y_order.get_shape(),reduced.get_shape())
        
        
        
        def custom_metric_accuracy(u,v):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                print("using custom metric.....")
                def get_response_array(x):
                    return x[:,:,-self.input_dim_order:]
                
                
                
                actual_response=Lambda(get_response_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
                print(actual_response.get_shape(),"actual response")
                def idx_max_prob(x):
                    max_idx= K.argmax(x,axis=-1)
#                         convert to one hot, to tackle,incorrect question argmax
                    one_hot_idx=tf.one_hot(max_idx,
                                self.input_dim_order,
                                on_value=1.0,
                                off_value=0.0,
                                axis=-1)
                    return one_hot_idx
        
                max_predicted_prob = Lambda(idx_max_prob,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(dense_out)
                print( max_predicted_prob.get_shape()," max_predicted_prob")
                # we use merging, we want to find how many 1-1 matchings are there
                merged_metric = multiply([actual_response,  max_predicted_prob])


                accuracy = K.mean(K.sum(merged_metric,axis=-1))
                print(accuracy.get_shape())
                return accuracy

        ## 7. Creates model object with specified input and output
        self.model = Model(inputs=x, outputs=reduced)

        ## 8. Compile model by assigning loss function for backpropagtion
        self.model.compile(optimizer=self.optimizer,
                           loss='binary_crossentropy',
                           metrics=['accuracy',custom_metric_accuracy])

        print('Summary of the model')
        self.model.summary()
        
    def train_on_batch(self, x_train,y_train,y_train_order):
# y_train_order: not used
        self.model.train_on_batch(x_train, y_train)




    def test_on_batch(self, x_val, y_val,y_train_order):
        
        # y_train_order: not used
        """
       Test the model on a single batch of samples
       :return: Scalar test loss (if the model has a single output and no metrics) or list of scalars (if the model has multiple outputs and/or metrics).
       The attribute model.metrics_names will give you the display labels for the scalar outputs.
       """
#         print(self.model.metrics_names)
        return self.model.test_on_batch(x_val, y_val)

    def predict(self, x_val,y_train_order):
#              # y_train_order: not used
        return self.model.predict_on_batch(x_val)



    
    


# train.py

In [8]:
import pandas as pd
import numpy as np
import json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import argparse

# from DKT import DKTnet
# from data_helper import load_data, one_hot, preprocess

def get_callbacks():
    '''
    Some callback functions that you may find useful.
    Please refer to https://keras.io/callbacks/ for more detailed explaination
    '''
    checkpoint = ModelCheckpoint('my_model', 
                                 monitor='val_loss',
                                 verbose=2, 
                                 save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=2)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=0, min_lr=1e-4)
    return [checkpoint, early_stopping, reduce_lr]



def batch_generator(skill_array, response_array, batch_size=64, shuffle=True):
    """
    return: batches of data from the original data set for training
    
    """
    skill_array, response_array, skill_response_array=preprocess(skill_array, response_array)
    sample_num = skill_array.shape[0]
    if shuffle:
        shuffled_indices = np.random.permutation(sample_num)
        skill_array = skill_array.copy()[shuffled_indices]
        response_array = response_array.copy()[shuffled_indices]
        skill_response_array = skill_response_array.copy()[shuffled_indices]
        print('Training set shuffled')
    for ndx in range(0, sample_num, batch_size):
        skill_array_batch = skill_array[ndx:min(ndx+batch_size, sample_num)]
        response_array_batch = response_array[ndx:min(ndx+batch_size, sample_num)]
        skill_response_array_batch = skill_response_array[ndx:min(ndx+batch_size, sample_num)]
        yield skill_response_array_batch, response_array_batch, skill_array_batch

def create_validation_data(skill_array, response_array,size=0.2):
    """
    return: split of data from the original data set for testing
    """

    sample_num = skill_array.shape[0]
    shuffled_indices = np.random.permutation(sample_num)
    skill_array = skill_array.copy()[shuffled_indices]
    response_array = response_array.copy()[shuffled_indices]
#     skill_response_array = skill_response_array.copy()[shuffled_indices]
    print('Data set shuffled, preparing for split')
    split_index=int(size*sample_num)
    train_index=sample_num-split_index
    print('train:', train_index, "test:",split_index)
    skill_array_train = skill_array[0:train_index]
    response_array_train = response_array[0:train_index]
#     skill_response_array_train = skill_response_array[0:train_index]
    
    skill_array_test = skill_array[train_index:]
    response_array_test = response_array[train_index:]
#     skill_response_array_test = skill_response_array[train_index:]
    
    return skill_array_train, response_array_train,skill_array_test,response_array_test


def train_on_batch(skill_array, response_array,custom_loss=False,):
    """This function creates a DKT MODEL object using DKT.py and
    trains it using the batched data"""

    input_dim = 2*336  #2* num_skills
#     input_dim_order = skill_array.shape[-1] #num_skills
    input_dim_order = 336
    hidden_layer_size = 40
    batch_size = 64
    epochs = 5
    print("batch size==",batch_size)
    print("epoch size=", epochs)
    print("input_dim_order",input_dim_order)
    print("input_dim",input_dim)
    

    '''
    parameters like batch_size, epochs x_train, y_train, y_train_order and validations are useless
    if you are doing a training by batch
    '''
    if custom_loss ==False:
        dkt = DKTnettest(input_dim, 
                    input_dim_order, 
                    hidden_layer_size, 
                    batch_size, 
                    epochs,
#                     x_train=skill_response_array[:, :, :], 
#                     y_train=response_array[:, :, np.newaxis], 
#                     y_train_order=skill_array[:, :, :],
                    validation_split=0.2,
                    validation_data=0.2,
                    optimizer='adam',
                    callbacks=None)
        
    else:
        dkt = DKTcustomloss(input_dim, 
                    input_dim_order, 
                    hidden_layer_size, 
                    batch_size, 
                    epochs,
#                     x_train=skill_response_array[:, :, :], 
#                     y_train=response_array[:, :, np.newaxis], 
#                     y_train_order=skill_array[:, :, :],
                    validation_split=0.2,
                    validation_data=0.2,
                    optimizer='adam',
                    callbacks=None)
        
        
    print('x_train.shape,y_train.shape')
   
    print(skill_array.shape,response_array.shape)
   
    dkt.build_train_on_batch()
    

    '''
    For simplification, we are over fitting on the training set here.
    In your model, you should do a train-test split or cross-validation which can be found in sklearn package.
    '''
    skill_array_train, response_array_train,skill_array_test,response_array_test=create_validation_data(skill_array, response_array,size=dkt.validation_split)



    for e in range(epochs):
        print('***Epoch', e+1, 'starts****')
        iteration = 0
        train_loss=0
        train_accuracy=0
        test_loss=0
        test_accuracy=0
        total_iteration_num = 1 + (skill_array.shape[0] - 1) // batch_size
        for skill_response_array_batch, response_array_batch, skill_array_batch in batch_generator(skill_array_train, response_array_train,  batch_size=batch_size):
            dkt.train_on_batch(skill_response_array_batch[:, :, :],response_array_batch[:, :, np.newaxis],  skill_array_batch[:, :, :])
            loss,acc=dkt.test_on_batch(skill_response_array_batch[:, :, :],response_array_batch[:, :, np.newaxis],  skill_array_batch[:, :, :])
            train_loss+=loss
            train_accuracy+=acc
            iteration += 1
        print('Evalutaion training data result,loss,accuracy', train_loss/iteration,train_accuracy/iteration)
        print("iter: {}/{} done".format(iteration, total_iteration_num))
        train_result=dkt.test_on_batch(skill_response_array_batch[:, :, :],response_array_batch[:, :, np.newaxis],  skill_array_batch[:, :, :])[0]
        test_batch_i=0
        for sr, r, s in batch_generator(skill_array_test, response_array_test,  batch_size=batch_size):
           
            l,a=dkt.test_on_batch(sr,r[:, :, np.newaxis], s)
            test_loss+=l
            test_accuracy+=a
            test_batch_i+=1

#         skill_array_test,response_array_test, skill_response_array_test=preprocess(skill_array_test,response_array_test)
#         result = dkt.test_on_batch(skill_response_array_test[:, :, :],response_array_test[:, :, np.newaxis],  skill_array_test[:, :, :])
#         print('Evalutaion training data result', train_result)
        print('Evalutaion validation data result, loss, accuracy', test_loss/test_batch_i,test_accuracy/test_batch_i)
        '''
        You should implement your own evaluation function here to evaluate your result on the validation set if each sample have different timesteps
        '''
#     prediction = dkt.predict(skill_response_test[0:1, :, :],
#                                 skill_array[0:1, 1:, :])

#     print('Check Prediction Output:', prediction,response_array[:, :, np.newaxis])
    # print('Check shape:Input,prediction, y_train:',dkt.x_train.shape, prediction.shape,dkt.y_train[0:1].shape)
    
    
    
    
    
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-S',"--skill_df", type=str, required=True)
#     parser.add_argument('-R',"--response_df",type=str, required=True)
#     parser.add_argument('-dict',"--skill_dict", type=str, required=True)
#     args = parser.parse_args()
#     print(args)

#     response_df,skill_df,skill_dict=load_data(args.skill_df,args.response_df,args.skill_dict)
#     # response_df,skill_df,skill_dict=load_data()
#     skills_num = len(skill_dict)
#     print('Number of skills are :{}'.format(skills_num))
#     skill_array, response_array, skill_response_array = preprocess(skill_df, response_df, skills_num)
#     # train(skill_array, response_array, skill_response_array)
#     train_on_batch(skill_array, response_array, skill_response_array)

#     # python train.py -S skill_df_delft_15.csv -R response_df_delft_15.csv -dict skill_dict_delft_15.json
    

In [9]:

# """phase 1- delft 15"""

data_dir1='code/delft15_phase1/'
event_dict=('code/delft15_phase2/'+'all_event_dict_delft15.json')
skill=os.path.join(data_dir1+"problemevent_df_delft_15.csv")
response=os.path.join(data_dir1+"problemresponse_df_delft_15.csv")


# subset for testing

skill_matrix,response_array,problem_num, non_problem_event_num=load_data(skill,response,event_dict,problem_num=336)



# skill_matrix=skill_matrix[:500,:]
# response_array=response_array[:500,:]

# # load preprocess in each batch, put in train_batch fynction
# print('Number of non-problem events:', non_problem_event_num)
# skill_array, response_array, skill_response_array=preprocess(skill_matrix,response_array,problem_num, non_problem_event_num)

train_on_batch(skill_matrix,response_array,custom_loss=True)

batch size== 64
epoch size= 5
input_dim_order 336
input_dim 672
DKTnet initialization done
x_train.shape,y_train.shape
(9242, 100) (9242, 100)
using custom loss.....
using custom loss.....
Summary of the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, None, 672)         0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 672)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 40)          114080    
_________________________________________________________________
dense_1 (Dense)              (None, None, 336)         13776     
Total params: 127,856
Trainable params: 127,856
Non-trainable params: 0
_________________________________________________________________
Data set shuffled, preparing for split
train: 7394 test: 1

In [ ]:
# train_on_batch(skill_array,response_array, skill_response_array,custom_loss=False)

In [ ]:
# train_on_batch(skill_array,response_array, skill_response_array)

In [ ]:

# phase 1 : is_behaviour=FALSE
skill_matrix,response_array,problem_num, non_problem_event_num=load_data(skill,response,event_dict,problem_num=336,is_behaviour=False)
skill_matrix=skill_matrix[:1000,:]
response_array=response_array[:1000,:]
print('Number of non-problem events:', non_problem_event_num)
skill_array, response_array, skill_response_array=preprocess(skill_matrix,response_array,problem_num, non_problem_event_num)

In [ ]:
train_on_batch(skill_array,response_array, skill_response_array,custom_loss=True)

In [ ]:
k=np.array([0,0,0,0,10,15,6,7]).reshape(2,2,2)
k

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()
s=(K.argmax(k,axis=-1))
s.eval().shape
o=tf.one_hot(
    s,
    4,
    on_value=3,
    off_value=0,
    axis=-1,
    dtype=None
)


In [ ]:
s.eval()

In [ ]:
o.eval()

In [ ]:
K.mean(K.sum(o,axis=-1)).eval()

In [ ]:
K.sum(o,axis=-1).eval()

In [ ]:
s=[0,0]
s+=[1,2]
s